In [3]:
import numpy as np

def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0,1,n_steps)
    series = 0.5 * np.sin((time-offsets1)*(freq1*10+10))
    series  +=0.2 * np.sin((time-offsets2)*(freq2*20+20))
    series  +=0.1* (np.random.rand(batch_size, n_steps)-0.5)
    return series[..., np.newaxis].astype(np.float32)

In [5]:
n_steps = 50
series = generate_time_series(10000, n_steps+1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [6]:
y_pred = X_valid[:,-1]

In [7]:
import tensorflow as tf
from tensorflow import keras

/Users/James/.pyenv/versions/3.8.5/envs/ml_env/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [8]:
#Naive forecasting
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

0.020470455

In [12]:
#Simple fully connected layer
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [50,1]), 
    keras.layers.Dense(1)
])

In [32]:
def compile_model(model = model):
    model.compile(loss = "mean_squared_error", optimizer = keras.optimizers.Adam())
    history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))
    evaluate = model.evaluate(X_test, y_test)
    return evaluate

In [18]:
model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 5ms/step - loss: 0.0040


0.003998058382421732

In [19]:
#Simple RNN
model = keras.models.Sequential([
    keras.layers.SimpleRNN(1, input_shape = [None, 1])
    
])

In [20]:
model.compile(loss = "mean_squared_error", optimizer = keras.optimizers.Adam())
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), verbose = 0)
model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 6ms/step - loss: 0.0135


0.013519316911697388

Worse than our fully connected model!

In [21]:
#Deep RNN
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20,return_sequences = True,  input_shape = [None, 1]), 
    keras.layers.SimpleRNN(20,return_sequences = True),
    keras.layers.SimpleRNN(1)
    
])

In [22]:
model.compile(loss = "mean_squared_error", optimizer = keras.optimizers.Adam())
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), verbose = 0)
model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 13ms/step - loss: 0.0032


0.0032497637439519167

Better than our fully connected model!

In [28]:
#Forecasting several time steps ahead
series = generate_time_series(1,n_steps + 10)
X_new, Y_new = series[:,:n_steps], series[:,n_steps]
X= X_new
for step_ahead in range(10):
    y_pred_one = model.predict(X[:,step_ahead:])[:,np.newaxis,:]
    X = np.concatenate([X, y_pred_one], axis = 1)
            
Y_pred = X[:,n_steps]

In [29]:
Y_pred

array([[-0.61122364]], dtype=float32)

In [41]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

In [42]:
X_train.shape, Y_train.shape

((7000, 50, 1), (7000, 50, 10))

In [45]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

model.compile(loss="mse", optimizer=keras.optimizers.Adam(lr=0.01), metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 10s 36ms/step - loss: 0.0774 - last_time_step_mse: 0.0697 - val_loss: 0.0433 - val_last_time_step_mse: 0.0314
Epoch 2/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0429 - last_time_step_mse: 0.0315 - val_loss: 0.0339 - val_last_time_step_mse: 0.0220
Epoch 3/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0331 - last_time_step_mse: 0.0210 - val_loss: 0.0303 - val_last_time_step_mse: 0.0179
Epoch 4/20
219/219 [==============================] - 7s 34ms/step - loss: 0.0281 - last_time_step_mse: 0.0153 - val_loss: 0.0247 - val_last_time_step_mse: 0.0119
Epoch 5/20
219/219 [==============================] - 7s 32ms/step - loss: 0.0246 - last_time_step_mse: 0.0118 - val_loss: 0.0225 - val_last_time_step_mse: 0.0095
Epoch 6/20
219/219 [==============================] - 7s 33ms/step - loss: 0.0235 - last_time_step_mse: 0.0111 - val_loss: 0.0221 - val_last_time_step_mse: 0.0101
Epoch 7/20
219/219 [=

In [46]:
#LSTM
model = keras.models.Sequential([
    keras.layers.LSTM(20, return_sequences = True, input_shape = [None, 1]), 
    keras.layers.LSTM(20, return_sequences = True), 
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [48]:
#Wavenet 

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape = [None, 1]))
for rate in (1,2,4,8) * 2:
    model.add(keras.layers.Conv1D(filters = 20, kernel_size = 2, padding = "causal", 
                                 activation = "relu", dilation_rate = rate))
    
model.add(keras.layers.Conv1D(filters = 10, kernel_size = 1))
model.compile(loss = "mse", optimizer = "adam", metrics = [last_time_step_mse])
history = model.fit(X_train, Y_train, epochs = 20, 
                   validation_data = (X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 5s 18ms/step - loss: 0.1003 - last_time_step_mse: 0.0921 - val_loss: 0.0363 - val_last_time_step_mse: 0.0233
Epoch 2/20
219/219 [==============================] - 4s 17ms/step - loss: 0.0336 - last_time_step_mse: 0.0201 - val_loss: 0.0301 - val_last_time_step_mse: 0.0167
Epoch 3/20
219/219 [==============================] - 4s 18ms/step - loss: 0.0300 - last_time_step_mse: 0.0167 - val_loss: 0.0285 - val_last_time_step_mse: 0.0153
Epoch 4/20
219/219 [==============================] - 7s 31ms/step - loss: 0.0280 - last_time_step_mse: 0.0150 - val_loss: 0.0262 - val_last_time_step_mse: 0.0136
Epoch 5/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0262 - last_time_step_mse: 0.0138 - val_loss: 0.0255 - val_last_time_step_mse: 0.0132
Epoch 6/20
219/219 [==============================] - 5s 21ms/step - loss: 0.0249 - last_time_step_mse: 0.0124 - val_loss: 0.0241 - val_last_time_step_mse: 0.0116
Epoch 7/20
219/219 [==